In [73]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from oasis.functions import deconvolve # install using conda install to avoid having to build
from scipy import signal
from dewan_calcium import DewanDeconv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
FRAMERATE=20

In [95]:
transients = DewanDeconv.load_transients('./traces.csv')
zscore_transients = DewanDeconv.z_score_data(transients)


In [96]:
from concurrent.futures import ProcessPoolExecutor

results = []

with ProcessPoolExecutor(max_workers=8) as pool:
    results = pool.map(DewanDeconv.smooth_data, zscore_transients.T.values)

array([-0.44032848, -0.55101156, -0.5782793 , ..., -0.3035942 ,
       -0.21498765, -0.22887805], dtype=float32)